# Extracting the Tangible Assets, and the year they relate to

## Method Overview

1.  Create a table of discovered sentences, inc. those that spread over two lines, and the coordinates of their bounding boxes.

2.  Identify isolated numeric values in the parsed text.

3.  For each value, subset the discovered sentences to those that are in-line with the value, and to the left.

4.  Somehow determine which of the numbers are those from the balance sheet, and what year they are for! (Work in progress)

5.  Separately extract years and their ordering, by looking for two year numbers in line with one year's difference and the older year on the right.  This is probably easier with this sparse table structure.

From UK National Accounts - A Short Guide:

"The national ccounts identifiy two types of asset within the economy. The first of
these is non-financial assets and includes fixed assets (buildings, vehicles,
machinery), valuables, inventories and non-produced assets (such as land). The
second type is financial assets. These include currency holdings, bank deposits,
ownership of shares and loans (from the point of view of the lender). Every financial
asset has an equal and opposite liability – in the examples above, the central bank is
liable for currency, the bank where the deposits are held, the share issuer and, in the
case of a loan, the borrower. "

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import pytesseract                            # API for letting python interface with Google's tesseract OCR software

import importlib

import xbrl_image_parser as xip


# -----------------------------------------------------------------
import re
import cv2                                    # Open Computer Vision library
import PyPDF2                                 # All things PDF format related
import io                                     # Something about messing with memory
from wand.image import Image                  # For messing with images
from PIL import Image as Im                   # Likewise images
import codecs                                 # Unknown

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.path import Path
from matplotlib.patches import PathPatch

from imageio import imread  # Lets me put an image into a plot

/home/martin/.local/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


## 1. Get the filenames of the example data for my convenience

In [2]:
# Get a list of all of the pdf files in the directory "CH_records"
files = [filename.split(".")[0] for filename in os.listdir("./working/ocr_output_compiled") if ".csv" in filename]

In [3]:
for each in range(len(files)):
    print(each, files[each])

0 00002404
1 868273
2 1983517
3 2765595
4 03293902
5 02959325
6 00542515
7 01539777
8 02714555
9 00030177
10 04802747
11 02266230
12 00983951
13 01002610
14 01804186
15 5508774
16 02430955
17 00053475
18 02245999
19 00553535
20 3387163
21 01337451
22 3459907
23 00178090
24 00468115
25 01369166
26 00782931
27 09457025
28 983951
29 01370175
30 06005142
31 04860660
32 2303730
33 02582534
34 00477955
35 04558828
36 06034603
37 3824626


## 2. Read in a csv file of data extracted from a PDF

Then perform some functions; calculating useful metadata about coordinates of discovered text, converting amenable text to numeric formats.

In [4]:
# Rediscovering what works...
importlib.reload(xip)
index=0

# So far can create all those extra geometric features, can convert to numeric
test = pd.read_csv("./working/ocr_output_compiled/"+files[index]+".csv")
line_test = xip.make_measurements(test)
line_test['numerical'] = xip.convert_to_numeric(line_test['text'])

# Drop all weird page-marking lines and such which have no text
line_test = line_test[line_test['conf'] != -1]

## 3. Create a table of discovered sentences and their coordinates

This table will contain all of the row labels from the balance sheet, as well as a LOT of other material

In [5]:
importlib.reload(xip)
#os.system("firefox " + "./example_data_PDF/" + files[index] + ".pdf" + " &")

agg_text = xip.aggregate_sentences_over_lines(line_test)

In [6]:
agg_text[agg_text['text'].apply(lambda x: "balancesheet" in x.lower())]

,block_num,bottom,csv_num,left,line_num,par_num,right,text,top
71,4.0,1297.0,3.0,306.0,3.0,3.0,563.0,balancesheet,1269.0
256,7.0,1611.0,8.0,301.0,1.0,1.0,821.0,postbalancesheetevents,1583.0
333,3.0,1206.0,11.0,308.0,2.0,1.0,2242.0,packetcompanylimitedfortheyearendeddecemberwhi...,1068.0
414,3.0,632.0,14.0,309.0,1.0,1.0,739.0,balancesheet,580.0
439,2.0,652.0,15.0,309.0,1.0,1.0,1110.0,balancesheetcontinued,580.0
629,16.0,2578.0,21.0,507.0,1.0,1.0,2308.0,assetsthataresubjecttodepreciationoramortisati...,2246.0
636,16.0,2723.0,21.0,507.0,8.0,1.0,2307.0,nonfinancialassetsthathavebeenpreviouslyimpair...,2585.0
650,11.0,1228.0,22.0,510.0,1.0,1.0,2308.0,ateachbalancesheetdatestocksareassessedforimpa...,1091.0
691,11.0,1805.0,23.0,498.0,1.0,1.0,2299.0,financialassetsandliabilitiesareoffsetandthene...,1667.0
699,15.0,2533.0,23.0,495.0,1.0,1.0,2298.0,derivativefinancialinstrumentsaremeasuredinthe...,2248.0


## 4. Find sentences/labels that are in-line with discovered numbers

This should, therefore, contain all variables from the balance sheet

In [7]:
# Drop non-numeric entries
numeric_df = line_test.loc[line_test['numerical'].dropna().index,]

results = pd.DataFrame()

# Find matches
for index, row in numeric_df.iterrows():
    
    # Filter to all text that is left of, and starts level to or above, the number
    matched_text = agg_text[agg_text['csv_num'] == row['csv_num']]     # Filter to same source page
    matched_text = matched_text[matched_text['top'] < row['bottom']]   # No elements that start below number on page
    matched_text = matched_text[matched_text['bottom'] > row['top']]
    matched_text = matched_text[matched_text['right'] < row['left']]   # No elements to the right of the number
    
    #print(len(matched_text))
    # Sort the matches by distance upwards from number
    matched_text = matched_text.sort_values('top', ascending=False)
    
    # Handles issue where, with only one row left in matched_text, it becomes a series
    if len(matched_text) > 0:
        try:
            row['matched_texts'] = matched_text.iloc[0,:]['text']
            #print(row['numerical'], matched_text.iloc[0,:]['text'])
        except:
            row['matched_texts'] = matched_text['text']
            #print(row['numerical'], matched_text['text'])
    
        results = results.append(row)

## 5. Final step;  Filtering!

This is quite complicated.  The above steps produce a giant table of text-number pairs, wherever they were found in-line.  This contains the variables we seek, but also a lot of random crap extracted from elsewhere in the documents.  To deal with this we'll clean the data through a number of arbitrary filters designed to remove spurious matches.

In [8]:
# Define a function for detecting if a string is our balance sheet title
check_bs = lambda x: np.where( re.search( "^[abbreviated]*balancesheet", x ), True, False)

# Get a list of pages the title string/regex is found on
BS_page_list = pd.unique(agg_text[agg_text['text'].apply(check_bs)]['csv_num'])

# Filter results to those pages
results = results[results['csv_num'].apply(lambda x: x in BS_page_list)]

In [9]:
# Find the column header "^notes*$" (numbers are references to notes on financial accounts)
check_notes = lambda x: np.where( re.search( "^notes*$", str(x).lower() ), True, False)

# Get a table of entries that are that column header
note_entries = agg_text[agg_text['text'].apply(check_notes)]

# List and then drop every numeric entry aligned to left of discovered column header
to_drop = []
for index, row in note_entries.iterrows():
    temp_df = results[results['csv_num'] == row['csv_num']] # Filter to entries on relevant page
    temp_df = temp_df[temp_df['left'] < row['left']+90]     # Find numbers vertically aligned or to left of identified header
    to_drop = to_drop + list(temp_df.index)

results = results.drop(to_drop)

In [274]:
note_entries

,block_num,bottom,csv_num,left,line_num,par_num,right,text,top
389,4.0,868.0,13.0,1647.0,1.0,1.0,1740.0,note,839.0
416,4.0,913.0,14.0,1126.0,1.0,1.0,1219.0,note,884.0
441,3.0,887.0,15.0,1125.0,2.0,1.0,2281.0,note,844.0
1243,6.0,1517.0,40.0,1107.0,1.0,1.0,1172.0,note,1496.0
1280,4.0,1000.0,41.0,1367.0,1.0,1.0,1459.0,note,971.0


In [10]:
# Filter numbers matched with "page" (page number!)
results = results[results['matched_texts'] != "page"]

# Filter numbers matched with "note" (column headers!)
results = results[results['matched_texts'].apply(lambda x: not np.where( re.search( "^notes*$", x ), True, False))]

In [11]:
# Filter out any page with the words "notes to the financial statements"
notes_page_list = pd.unique(agg_text[agg_text['text'].apply(lambda x: "notestothefinancialstatements" in x)]['csv_num'])

results = results[results['csv_num'].apply(lambda x: x not in notes_page_list)]

In [12]:
results[['text', 'numerical', 'matched_texts']]

,text,numerical,matched_texts
4760,180,180.0,intangibleassets
4762,"28,271",28271.0,tangibleassets
4770,446,446.0,stocks
4772,"23,174",23174.0,debtorsamountsfallingduewithinoneyear
4774,"10,208",10208.0,cashatbankandinhand
4780,"(10,936)",10936.0,creditorsamountsfallingduewithinoneyear
4782,"22,892",22892.0,netcurrentassets
4784,"51,346",51346.0,totalassetslesscurrentliabilities
4788,"(2,620)",2620.0,creditorsamountsfallingdueaftermorethanoneyear
4792,(18),18.0,pensions


## 6. Get an example of a variable

For "creditors", we get all four relevant numbers from the balance sheet and no other (spurious) variables from elsewhere.  Likewise for "tangible".  The left-most value for each pair of variables (the one with the lowers "left" coordinate) will be that of the most recent year.

In [13]:
results[results['matched_texts'].apply(lambda x: "credit" in x)][['csv_num', 'numerical', 'matched_texts', 'left', 'top']]

,csv_num,numerical,matched_texts,left,top
4780,14.0,10936.0,creditorsamountsfallingduewithinoneyear,1362.0,1657.0
4788,14.0,2620.0,creditorsamountsfallingdueaftermorethanoneyear,1640.0,1962.0
4830,14.0,7527.0,creditorsamountsfallingduewithinoneyear,1912.0,1658.0
4840,14.0,3071.0,creditorsamountsfallingdueaftermorethanoneyear,2173.0,1963.0


In [14]:
results[results['matched_texts'].apply(lambda x: "tangible" in x)][['csv_num', 'numerical', 'matched_texts', 'left', 'top']]

,csv_num,numerical,matched_texts,left,top
4760,14.0,180.0,intangibleassets,1693.0,1025.0
4762,14.0,28271.0,tangibleassets,1639.0,1087.0
4808,14.0,137.0,intangibleassets,2219.0,1027.0
4810,14.0,28681.0,tangibleassets,2166.0,1088.0


### How many unique labels are left?

In [15]:
print(len(results['matched_texts'].unique()))

results.groupby('matched_texts').agg('count')['word_num']

16


matched_texts
calledupsharecapital                              2
cashatbankandinhand                               2
creditorsamountsfallingdueaftermorethanoneyear    2
creditorsamountsfallingduewithinoneyear           2
debtorsamountsfallingduewithinoneyear             2
hedgingreserve                                    2
intangibleassets                                  2
netassets                                         2
netcurrentassets                                  2
pensionliability                                  2
pensions                                          2
profitandlossaccount                              2
revaluationreserve                                2
stocks                                            2
tangibleassets                                    2
totalassetslesscurrentliabilities                 2
Name: word_num, dtype: int64

In [16]:
# Did we get the right pages?
pd.unique(results['csv_num'])

array([14., 15.])

# PLOTTING DIAGNOSTICS - Finding out if the blocking algorithm is truly unreliable

In [17]:
# Figure out the ratio of whitespace in each block

def calculate_whitespace(data, threshold=0.8):
    
    # to hold calculated values
    whitespace = []
    block_num = []
    csv_num = []
    whitespace_frac = []
    whitespace_anomalous = []

    for csv in data['csv_num'].unique():
        for block in data[data['csv_num']==csv]['block_num'].unique():
            
            subset = data[(data['csv_num']==csv) & (data['block_num']==block)]      # Get the relevant data
            total_area = subset[subset['par_num']==0]['area'].sum()                 # Total area of block
            word_area = subset[subset['word_num'] > 0]['area'].sum()                # Total area occupied by words (assume rectangles)
            whitespace.append(total_area - word_area)                               # Therefore total area whitespace
            csv_num.append(csv)
            block_num.append(block)
            fraction = total_area - word_area / total_area                     # Finally, fraction whitespace
            whitespace_frac.append(fraction)   
            
            # Decide if it's anomalous or not following threshold (between 0 and 0.99)
            whitespace_anomalous.append((fraction > threshold) & (fraction < 0.99 ))
            
    whitespace_data = pd.DataFrame({"whitespace":whitespace,
                                    "whitespace_frac":whitespace_frac,
                                    "whitespace_anomalous":whitespace_anomalous,
                                    "block_num":block_num,
                                    "csv_num":csv_num})
    
    return( data.merge(whitespace_data, on=["csv_num", "block_num"], how="left") )

In [18]:
# Plotting commands

def create_path_patches(line_data, csv_num):
    
    line_data=line_data[line_data['csv_num']==csv_num].reset_index()
    
    # For storing all of the created pathpatches
    path_patches = []

    # Need the height and width of the image to create the canvas
    image_length = line_data.loc[0]['height']
    image_width = line_data.loc[0]['width']

    #sample = line_data[line_data['line_num'] == 0]
    sample = line_data
    sample = sample.drop(0)
    # Iterate through the samples
    for index, row in sample.iterrows():
    
        # Define the shape by specifying its corners, remember
        # x-y coords start at BOTTOM LEFT of image
        vertices = [(row['left'], row['top']),      # Top left
                     (row['left'], row['bottom']),   # Bottom left
                     (row['right'], row['bottom']),  # Bottom right
                     (row['right'], row['top']),     # Top right
                     (0, 0)]                                        # Blank for closing off the polygon automatically
        # Codify instructions to path tracer for 
        codes = [Path.MOVETO] + [Path.LINETO]*3 + [Path.CLOSEPOLY]
    
        # Face colour, based on whether the whitespace is anomalous
        face_colour = str(np.where(row['whitespace_anomalous']==True, "red", "green"))
    
        # Line colour, based on whether the word spacing is anomalous
        #line_colour = str(np.where(row['word_spacing_anomalous']==True, "red", "green"))

        # Convert gathered information to a path
        path_patches.append( {"path":Path(vertices, codes),
                              "face_colour":face_colour,
                              "line_colour": "green"} )
    
    return(path_patches, vertices)

In [19]:
def plot_doc(path_patches, vertices, filepath):

    # Plot the lot!
    fig, ax = plt.subplots(figsize=(270, 100))
    for each in path_patches:
        ax.add_patch(PathPatch(each['path'],
                               facecolor = each['face_colour'],
                               edgecolor=each['line_colour'],
                               lw=3,
                               alpha=0.3))
    
    ax.set_title("Discovered blocks")

    ax.dataLim.update_from_data_xy(vertices)
    ax.autoscale_view()

    plt.imshow(imread(filepath), cmap="gray")
    plt.savefig("./output_graphical/discovered_blocks.png")
    plt.show()

In [ ]:
# Pick the file and page to plot
file_index=1
csv_num=15

# Pick the block level to portray
level=4

# Do all the maths
white_test=calculate_whitespace(xip.make_measurements(pd.read_csv("./working/ocr_output_compiled/"+files[file_index]+".csv")))

# Define the boxes to plot
path_patches, vertices = create_path_patches(white_test[white_test['level']==level], csv_num=csv_num)

# Draw everything up for inspection
plot_doc(path_patches, vertices, "./working/preprocessed/"+files[file_index]+"-"+str(csv_num)+".png")